In [27]:
import json

# Fungsi escape karakter untuk SQL
def escape_sql(value):
    if value is None or value == "":
        return "NULL"
    if isinstance(value, str):
        return "'" + value.replace("'", "''") + "'"
    return str(value)

# Load JSON
with open('data/json/regulation_data_final.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

queries = []  # INSERT utama
relation_queries = []  # INSERT relasi pasal dan regulasi

# Kumpulkan semua ID artikel dulu agar bisa validasi relasi
all_article_ids = set()
for reg in data:
    for art in reg.get('content', {}).get('articles', {}).values():
        all_article_ids.add(art['id'])

for reg in data:
    # REGULATIONS
    queries.append(f"""
INSERT INTO regulations (id, url, download_link, download_name, title, about, type, short_type, amendment, number, year, institution, issue_place, issue_date, effective_date) 
VALUES (
    {escape_sql(reg['id'])}, 
    {escape_sql(reg['url'])}, 
    {escape_sql(reg['download_link'])}, 
    {escape_sql(reg['download_name'])}, 
    {escape_sql(reg['title'])}, 
    {escape_sql(reg['about'])}, 
    {escape_sql(reg['type'])}, 
    {escape_sql(reg['short_type'])}, 
    {escape_sql(reg['amendment'])}, 
    {escape_sql(reg['number'])}, 
    {reg['year']}, 
    {escape_sql(reg['institution'])}, 
    {escape_sql(reg['issue_place'])}, 
    {escape_sql(reg['issue_date'])}, 
    {escape_sql(reg['effective_date'])}
);
""")

    # SUBJECTS
    for subject in reg.get('subjects', []):
        queries.append(f"INSERT INTO subjects (id, subject) VALUES ({escape_sql(reg['id'])}, {escape_sql(subject)});")

    # STATUS
    status = reg.get('status', {})
    repealed = ','.join(status.get('repealed', []))
    repeal = ','.join(status.get('repeal', []))
    amended = ','.join(status.get('amended', []))
    amend = ','.join(status.get('amend', []))

    queries.append(f"""
INSERT INTO status (id, repealed, repeal, amended, amend) 
VALUES (
    {escape_sql(reg['id'])}, 
    {escape_sql(repealed)}, 
    {escape_sql(repeal)}, 
    {escape_sql(amended)}, 
    {escape_sql(amend)}
);
""")

    # REGULATION RELATIONS (skip kalau isinya bukan ID angka)
    for amended_id in status.get('amended', []):
        if amended_id.isdigit():
            relation_queries.append(f"INSERT INTO regulation_relations (from_regulation_id, to_regulation_id, relation_type) VALUES ({escape_sql(reg['id'])}, {escape_sql(amended_id)}, 'mengubah');")

    for amend_id in status.get('amend', []):
        if amend_id.isdigit():
            relation_queries.append(f"INSERT INTO regulation_relations (from_regulation_id, to_regulation_id, relation_type) VALUES ({escape_sql(reg['id'])}, {escape_sql(amend_id)}, 'diubah_oleh');")

    for repealed_id in status.get('repealed', []):
        if repealed_id.isdigit():
            relation_queries.append(f"INSERT INTO regulation_relations (from_regulation_id, to_regulation_id, relation_type) VALUES ({escape_sql(reg['id'])}, {escape_sql(repealed_id)}, 'mencabut');")

    for repeal_id in status.get('repeal', []):
        if repeal_id.isdigit():
            relation_queries.append(f"INSERT INTO regulation_relations (from_regulation_id, to_regulation_id, relation_type) VALUES ({escape_sql(reg['id'])}, {escape_sql(repeal_id)}, 'dicabut_oleh');")

    # ARTICLES + ARTICLE RELATIONS
    articles = reg.get('content', {}).get('articles', {})
    for _, article in articles.items():
        queries.append(f"""
INSERT INTO articles (id, regulation_id, chapter_number, chapter_about, article_number, text, status) 
VALUES (
    {escape_sql(article['id'])}, 
    {escape_sql(reg['id'])}, 
    {escape_sql(article['chapter_number'])}, 
    {escape_sql(article['chapter_about'])}, 
    {escape_sql(article['article_number'])}, 
    {escape_sql(article['text'])}, 
    'Effective'
);
""")

        # ARTICLE RELATIONS
        if article.get('previous_article') and article['previous_article'] in all_article_ids:
            relation_queries.append(f"INSERT INTO article_relations (from_article_id, to_article_id, relation_type) VALUES ({escape_sql(article['previous_article'])}, {escape_sql(article['id'])}, 'berikutnya');")
            relation_queries.append(f"INSERT INTO article_relations (from_article_id, to_article_id, relation_type) VALUES ({escape_sql(article['id'])}, {escape_sql(article['previous_article'])}, 'sebelumnya');")

        for ref in article.get('references', []):
            if ref in all_article_ids:
                relation_queries.append(f"INSERT INTO article_relations (from_article_id, to_article_id, relation_type) VALUES ({escape_sql(article['id'])}, {escape_sql(ref)}, 'merujuk');")

        for amend in article.get('amend', []):
            if amend in all_article_ids:
                relation_queries.append(f"INSERT INTO article_relations (from_article_id, to_article_id, relation_type) VALUES ({escape_sql(article['id'])}, {escape_sql(amend)}, 'mengubah');")

    # DEFINITIONS
    definitions = reg.get('content', {}).get('definitions', [])
    for definition in definitions:
        queries.append(f"""
INSERT INTO definitions (id, regulation_id, name, definition) 
VALUES (
    {escape_sql(definition['id'])}, 
    {escape_sql(reg['id'])}, 
    {escape_sql(definition['name'])}, 
    {escape_sql(definition['definition'])}
);
""")

# Save ke file SQL
with open('seeding_output.sqlv3', 'w', encoding='utf-8') as f:
    for q in queries:
        f.write(q + '\n')
    f.write('\n-- INSERT RELASI PASAL DAN REGULASI\n')
    for rq in relation_queries:
        f.write(rq + '\n')

print(f"Generated {len(queries)} main data inserts and {len(relation_queries)} relation inserts successfully into 'seeding_output.sqlv3'!")


Generated 3623 main data inserts and 6372 relation inserts successfully into 'seeding_output.sqlv3'!


In [1]:
import json

# Fungsi escape karakter untuk SQL
def escape_sql(value):
    if value is None or value == "":
        return "NULL"
    if isinstance(value, str):
        return "'" + value.replace("'", "''") + "'"
    return str(value)

# Load JSON
with open('data/json/regulation_data_final.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

queries = []  # INSERT utama
relation_queries = []  # INSERT relasi pasal dan regulasi
relation_set = set()  # Hindari duplikat

# Kumpulkan semua ID artikel dulu agar bisa validasi relasi
all_article_ids = set()
reverse_next_index = {}
for reg in data:
    for art in reg.get('content', {}).get('articles', {}).values():
        all_article_ids.add(art['id'])
        next_id = art.get('next_article')
        if next_id:
            reverse_next_index.setdefault(next_id, []).append(art['id'])

for reg in data:
    # REGULATIONS
    queries.append(f"""
INSERT INTO regulations (id, url, download_link, download_name, title, about, type, short_type, amendment, number, year, institution, issue_place, issue_date, effective_date) 
VALUES (
    {escape_sql(reg['id'])}, 
    {escape_sql(reg['url'])}, 
    {escape_sql(reg['download_link'])}, 
    {escape_sql(reg['download_name'])}, 
    {escape_sql(reg['title'])}, 
    {escape_sql(reg['about'])}, 
    {escape_sql(reg['type'])}, 
    {escape_sql(reg['short_type'])}, 
    {escape_sql(reg['amendment'])}, 
    {escape_sql(reg['number'])}, 
    {reg['year']}, 
    {escape_sql(reg['institution'])}, 
    {escape_sql(reg['issue_place'])}, 
    {escape_sql(reg['issue_date'])}, 
    {escape_sql(reg['effective_date'])}
);
""")

    # SUBJECTS
    for subject in reg.get('subjects', []):
        queries.append(f"INSERT INTO subjects (id, subject) VALUES ({escape_sql(reg['id'])}, {escape_sql(subject)});")

    # STATUS
    status = reg.get('status', {})
    repealed = ','.join(status.get('repealed', []))
    repeal = ','.join(status.get('repeal', []))
    amended = ','.join(status.get('amended', []))
    amend = ','.join(status.get('amend', []))

    queries.append(f"""
INSERT INTO status (id, repealed, repeal, amended, amend) 
VALUES (
    {escape_sql(reg['id'])}, 
    {escape_sql(repealed)}, 
    {escape_sql(repeal)}, 
    {escape_sql(amended)}, 
    {escape_sql(amend)}
);
""")

    # REGULATION RELATIONS (skip kalau isinya bukan ID angka)
    for amended_id in status.get('amended', []):
        if amended_id.isdigit():
            relation_queries.append(f"INSERT INTO regulation_relations (from_regulation_id, to_regulation_id, relation_type) VALUES ({escape_sql(reg['id'])}, {escape_sql(amended_id)}, 'mengubah');")

    for amend_id in status.get('amend', []):
        if amend_id.isdigit():
            relation_queries.append(f"INSERT INTO regulation_relations (from_regulation_id, to_regulation_id, relation_type) VALUES ({escape_sql(reg['id'])}, {escape_sql(amend_id)}, 'diubah_oleh');")

    for repealed_id in status.get('repealed', []):
        if repealed_id.isdigit():
            relation_queries.append(f"INSERT INTO regulation_relations (from_regulation_id, to_regulation_id, relation_type) VALUES ({escape_sql(reg['id'])}, {escape_sql(repealed_id)}, 'mencabut');")

    for repeal_id in status.get('repeal', []):
        if repeal_id.isdigit():
            relation_queries.append(f"INSERT INTO regulation_relations (from_regulation_id, to_regulation_id, relation_type) VALUES ({escape_sql(reg['id'])}, {escape_sql(repeal_id)}, 'dicabut_oleh');")

    # ARTICLES + ARTICLE RELATIONS
    articles = reg.get('content', {}).get('articles', {})
    for _, article in articles.items():
        queries.append(f"""
INSERT INTO articles (id, regulation_id, chapter_number, chapter_about, article_number, text, status) 
VALUES (
    {escape_sql(article['id'])}, 
    {escape_sql(reg['id'])}, 
    {escape_sql(article['chapter_number'])}, 
    {escape_sql(article['chapter_about'])}, 
    {escape_sql(article['article_number'])}, 
    {escape_sql(article['text'])}, 
    'Effective'
);
""")

        def add_relation(from_id, to_id, relation_type):
            rel_key = (from_id, to_id, relation_type)
            if rel_key not in relation_set:
                relation_queries.append(f"INSERT INTO article_relations (from_article_id, to_article_id, relation_type) VALUES ({escape_sql(from_id)}, {escape_sql(to_id)}, '{relation_type}');")
                relation_set.add(rel_key)

        # ARTICLE RELATIONS - infer dari reverse next_article
        for prev_id in reverse_next_index.get(article['id'], []):
            if prev_id in all_article_ids:
                add_relation(prev_id, article['id'], 'berikutnya')
                add_relation(article['id'], prev_id, 'sebelumnya')

        # fallback single previous_article
        if article.get('previous_article') and article['previous_article'] in all_article_ids:
            add_relation(article['previous_article'], article['id'], 'berikutnya')
            add_relation(article['id'], article['previous_article'], 'sebelumnya')

        for ref in article.get('references', []):
            if ref in all_article_ids:
                add_relation(article['id'], ref, 'merujuk')

        for amend in article.get('amend', []):
            if amend in all_article_ids:
                add_relation(article['id'], amend, 'mengubah')

    # DEFINITIONS
    definitions = reg.get('content', {}).get('definitions', [])
    for definition in definitions:
        queries.append(f"""
INSERT INTO definitions (id, regulation_id, name, definition) 
VALUES (
    {escape_sql(definition['id'])}, 
    {escape_sql(reg['id'])}, 
    {escape_sql(definition['name'])}, 
    {escape_sql(definition['definition'])}
);
""")

# Save ke file SQL
with open('seeding_outputv4.sql', 'w', encoding='utf-8') as f:
    for q in queries:
        f.write(q + '\n')
    f.write('\n-- INSERT RELASI PASAL DAN REGULASI\n')
    for rq in relation_queries:
        f.write(rq + '\n')

print(f"Generated {len(queries)} main data inserts and {len(relation_queries)} relation inserts successfully into 'seeding_outputv4.sql'!")


Generated 3623 main data inserts and 6476 relation inserts successfully into 'seeding_outputv4.sql'!


In [5]:
import json

# Escape SQL
def escape_sql(value):
    if value is None or value == "":
        return "NULL"
    return "'" + value.replace("'", "''") + "'"

# Load JSON
with open('data/json/regulation_data_final.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

update_queries = []

for reg in data:
    reg_id = reg['id']
    content = reg.get('content', {})
    consideration_text = content.get('consideration', {}).get('text', '')
    observation_text = content.get('observation', {}).get('text', '')

    if consideration_text or observation_text:
        query = f"""
UPDATE regulations
SET
    consideration = {escape_sql(consideration_text)},
    observation = {escape_sql(observation_text)}
WHERE id = {escape_sql(reg_id)};
"""
        update_queries.append(query)

# Tulis ke file SQL
with open('seeding_consideration_observation.sql', 'w', encoding='utf-8') as f:
    for q in update_queries:
        f.write(q + "\n")

print(f"✅ Generated {len(update_queries)} UPDATE queries ke 'seeding_consideration_observation.sql'")


✅ Generated 63 UPDATE queries ke 'seeding_consideration_observation.sql'


In [2]:
import json

def escape_sql(value):
    if value is None or value == "":
        return "NULL"
    return "'" + value.replace("'", "''") + "'"

# Load JSON
with open('data/json/regulation_data_final.json', 'r', encoding='utf-8') as f:
    data = json.load(f)

update_queries = []

for reg in data:
    reg_id = reg.get("id")
    short_type = reg.get("short_type", "")
    number = reg.get("number", "")
    year = reg.get("year", "")
    articles = reg.get("content", {}).get("articles", {})

    for _, article in articles.items():
        article_id = article.get("id")
        article_number = article.get("article_number", "")
        title = f"Pasal {article_number} {short_type} Nomor {number} Tahun {year}"
        update_query = f"UPDATE articles SET title = {escape_sql(title)} WHERE id = {escape_sql(article_id)};"
        update_queries.append(update_query)

# Tulis ke file
with open("update_article_titles.sql", "w", encoding="utf-8") as f:
    for q in update_queries:
        f.write(q + "\n")

print(f"Generated {len(update_queries)} UPDATE queries into 'update_article_titles.sql'")


Generated 2423 UPDATE queries into 'update_article_titles.sql'
